Importing libraries

In [1]:
import pandas as pd
import re
import string
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

Loading the dataset

In [2]:
data = pd.read_csv('/content/tweets.csv')

In [3]:
data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [4]:
pd.set_option('display.max_colwidth',None)

In [5]:
data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


Dataset preprocessing

In [6]:
data = data.drop('id',axis =1)

Separating positive and negative tweets

In [7]:
data_pos = data[data['label']==1]
data_neg = data[data['label']==0]

Combining positive and negative tweets

In [8]:
data = pd.concat([data_pos,data_neg])

In [9]:
data.shape

(7920, 2)

In [10]:
data['label'].value_counts()

0    5894
1    2026
Name: label, dtype: int64

Cleaning and removing URLs

In [11]:
def remove_URL(text):
  return re.sub(r"http\S+", "", text)

Puncuation removal

In [12]:
punctuationfree = ''
def remove_pun(text):
  punc_free = ''.join([i for i in text if i not in string.punctuation])
  return punc_free

Word tokenization

In [13]:
def tokenize(text):
  word = nltk.word_tokenize(text)
  return word

Removing stopwords

In [14]:
def remove_stopwords(text):
  output = [i for i in text if i not in stopwords]
  return output

Lemmatization

In [15]:
lemma = WordNetLemmatizer()
def lemmat(text):
  lemm_text = [lemma.lemmatize(x) for x in text]
  return lemm_text

In [16]:
def preprocess(df_col):
  corpus = []
  for item in df_col:
    clean_msg = remove_pun(item)
    clean_msg = remove_URL(clean_msg)
    clean_msg = clean_msg.lower()
    clean_msg = tokenize(clean_msg)
    clean_msg = remove_stopwords(clean_msg)
    clean_msg = lemmat(clean_msg)
    corpus.append(' '.join(str(x)for x in clean_msg))
  return corpus

In [17]:
corpus = preprocess(data['tweet'])

In [18]:
corpus

['amazing service apple wont even talk question unless pay 1995 stupid support',
 'iphone software update fucked phone big time stupid iphones',
 'hey apple make new ipod dont make new color 2inches thinner make crash every five fuckin minite',
 'ha heavy machinery need apple really dropped ball design drinkyourhaterade',
 'contemplating giving iphone bandwagon simply cellcom new android depressing idontwantto',
 'shaqlockholmes samlouise1991 battery painful charge overnight lunchtime battery dead hateorange',
 'reason dont one iphone suck apple truth truthbetold agree fact realitycheck blackberrypictwittercomzpggd7cazn',
 'apple store gunna 5c screen monday ur fucking apple store fuckingpissed',
 'ive new ipod four day button back broke accidentally hit',
 'iphone screen go completely black keep getting text day cant fucking check apple 6thiphone',
 'ipod messed havent done nothing myloss apple',
 'ipod touch frozen apple logo start lose demis music pic kill someone soangry hateapple'

Bag of words

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1,2))
traindata= cv.fit_transform(corpus)

Splitting to x & y

In [20]:
x = traindata
y = data['label']

In [21]:
y

4       1
5       1
10      1
11      1
12      1
       ..
7915    0
7916    0
7917    0
7918    0
7919    0
Name: label, Length: 7920, dtype: int64

In [22]:
x

<7920x75853 sparse matrix of type '<class 'numpy.int64'>'
	with 172063 stored elements in Compressed Sparse Row format>

Train-Test splitting of data

In [23]:
from sklearn .model_selection import train_test_split
x_train ,x_test, y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42)

In [24]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_test,y_test)

RandomForestClassifier()

Finiding the Accuracy score

In [25]:
from sklearn.metrics import accuracy_score
y_pred = rf.predict(x_test)
accuracy_score(y_pred,y_test)

1.0